In [1]:
from transformer_lens import HookedTransformer
from neuron_text_simplifier import NeuronTextSimplifier
import torch as th

device = "cuda:2" if th.cuda.is_available() else "cpu"
# device = "cpu"
model_name = "EleutherAI/pythia-70m-deduped"
model = HookedTransformer.from_pretrained(model_name, device=device)
layer = 1
neuron = 1306
neurons = [[924],[1306]]
simplifier = NeuronTextSimplifier(model, layer, neuron)

/home/mchorse/miniconda3/envs/logan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


In [2]:
from circuitsvis.activations import text_neuron_activations
text = [
    " Yes Yes",
    "?\n Yes",
    "? \" \" Yes",
]
neuron = 1306
noise_level = 1.0

# def add_noise_to_text(text, noise_level):
#     if isinstance(text, str):
#         text = [text]
#     text_list = []
#     activation_list = []
#     for t in text:
#         split_text = model.to_str_tokens(t, prepend_bos=False)
#         tokens = model.to_tokens(t, prepend_bos=False)
#         # Add gaussian noise to the input of each word in turn, getting the diff in final neuron's response
#         embedded_tokens = self.model.embed(tokens)
#         batch_size, seq_size, embedding_size = embedded_tokens.shape
#         noise = th.randn(1, embedding_size, device=device)*noise_level
#         original = self.embedded_forward(embedded_tokens)[:,-1,self.neuron]
#         changed_activations = th.zeros(seq_size, device=device)
#         for i in range(seq_size):
#             embedded_tokens[:,i,:] += noise
#             neuron_response = self.embedded_forward(embedded_tokens)
#             changed_activations[i] = neuron_response[:,-1,self.neuron].item()
#             embedded_tokens[:,i,:] -= noise
#         changed_activations -= original
#         text_list += [x.replace('\n', '\\newline') for x in split_text] + ["\n"]
#         activation_list += changed_activations.tolist() + [0.0]
#     activation_list = th.tensor(activation_list).reshape(-1,1,1)
#     text_neuron_activations(tokens=text_list, activations=activation_list)

simplifier.add_noise_to_text(text, noise_level)

In [2]:
# Add gaussian noise to the input of each word in turn, getting the final neuron's response
token = model.to_tokens("? \" \" Yes")
embedded_tokens = simplifier.model.embed(token)
# add noise to the first word
batch_size, seq_size, embedding_size = embedded_tokens.shape
noise_level = 0.1
noise = th.randn(1, embedding_size, device=device)*noise_level
for i in range(seq_size):
    embedded_tokens[:,i,:] += noise
    neuron_response = simplifier.embedded_forward(embedded_tokens)
    print(neuron_response[:,-1,1306].item())
    embedded_tokens[:,i,:] -= noise

4.5115275382995605
2.574535846710205
4.468333721160889
3.9256157875061035
0.44166746735572815


In [196]:
neuron_response.shape

torch.Size([3, 5, 2048])

In [189]:
# https://github.com/neelnanda-io/TransformerLens/blob/main/transformer_lens/HookedTransformer.py#L370
from transformer_lens import HookedTransformer
model = HookedTransformer.from_pretrained(model_name, device="cuda:2")
token = model.to_tokens("testing 1 2 3")
logits, cache = model.run_with_cache(token)
print(logits.get_device())
print(model.cfg.device)
print(cache["blocks.0.mlp.hook_post"].get_device())

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer
2
cuda:2
0


In [143]:
from jaxtyping import Float, Int
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)
from circuitsvis.activations import text_neuron_activations
simplifier = NeuronTextSimplifier(model, layer, neuron)
def ablate_mlp_neurons(tokens, neurons: th.Tensor):
    def mlp_ablation_hook(
        value: Float[th.Tensor, "batch pos d_mlp"],
        hook: HookPoint
    ) -> Float[th.Tensor, "batch pos d_mlp"]:
        value[:, :, neurons] = 0
        return value
    return model.run_with_hooks(tokens, fwd_hooks=[(f"blocks.{simplifier.layer}.mlp.hook_post", mlp_ablation_hook)])
def visualize_logit_diff(text, neurons: th.Tensor, setting="true_tokens"):
    if isinstance(text, str):
        text = [text]
    text_list = []
    logit_list = []
    for t in text:
        split_text = model.to_str_tokens(t, prepend_bos=False)
        tokens = model.to_tokens(t, prepend_bos=False)
        original_logits = model(tokens)
        ablated_logits = ablate_mlp_neurons(tokens, neurons)
        diff_logits = original_logits - ablated_logits
        if setting == "true_tokens":
            # Gather the logits for the true tokens
            diff = diff_logits.gather(2,tokens.unsqueeze(2)).squeeze()
        elif setting == "max":
            diff = diff_logits.argmax(2)
        text_list += split_text + ["\n"]
        logit_list += diff.tolist() + [0.0]
    logit_list = th.tensor(logit_list).reshape(-1,1,1)
    print(f"Max & Min logit-diff: {logit_list.max().item():.2f} & {logit_list.min().item():.2f}")
    return text_neuron_activations(tokens=text_list, activations=logit_list)
# def visualize_logit_diff(tokens, neurons: th.Tensor, setting="true_tokens"):
setting = "true_tokens"
tokens = model.to_tokens("testing 1 2 3", prepend_bos=False)

neurons = th.tensor([[924],[1306]])
# neurons = th.tensor([[0]])

text = [
    "testing 1 2 3",
    " a b c",
    " d e f",
    " g h",
]


Max & Min logit-diff: 0.02 & -0.01


In [139]:
text_list

[['testing', ' 1', ' 2', ' 3'],
 [' a', ' b', ' c'],
 [' d', ' e', ' f'],
 [' g', ' h'],
 '\n',
 ['testing', ' 1', ' 2', ' 3'],
 [' a', ' b', ' c'],
 [' d', ' e', ' f'],
 [' g', ' h'],
 '\n',
 ['testing', ' 1', ' 2', ' 3'],
 [' a', ' b', ' c'],
 [' d', ' e', ' f'],
 [' g', ' h'],
 '\n',
 ['testing', ' 1', ' 2', ' 3'],
 [' a', ' b', ' c'],
 [' d', ' e', ' f'],
 [' g', ' h'],
 '\n']

In [122]:
from circuitsvis.tokens import colored_tokens
colored_tokens(["hey"], [1.3])

In [101]:
original_logits.gather(2,tokens[:,1:].unsqueeze(2)).squeeze()

tensor([-1.3670,  5.0511,  7.9925, 12.5219], device='cuda:2')

In [96]:
tokens

tensor([[    0, 19462,   337,   374,   495]], device='cuda:2')

In [63]:
# I want to clamp the activation of the neuron to, say 0.

neurons = th.tensor([[924],[1306]])
text = "Q:\n\n What is the capital of France?\n\n Paris."
tokens = model.to_tokens(text)
original_logits, cache = model.run_with_cache(tokens)
original_neuron_activation = cache["blocks.1.mlp.hook_post"].to(device)
# def mlp_ablation_hook(
#     value: Float[th.Tensor, "batch pos d_mlp"],
#     hook: HookPoint
# ) -> Float[th.Tensor, "batch pos d_mlp"]:
#     print(f"Shape of the value tensor: {value.shape}")
#     value[:, :, neurons] = 0.
#     return value
text = " 1 2 3 4 5 6 7 8 9 10 11 12 13"
tokens = model.to_tokens(text)
patching_logits, cache = model.run_with_cache(tokens)
patching_neuron_activation = cache["blocks.0.mlp.hook_post"].to(device)



def mlp_ablation_hook(
    value: Float[th.Tensor, "batch pos d_mlp"],
    hook: HookPoint
) -> Float[th.Tensor, "batch pos d_mlp"]:
    print(f"Shape of the value tensor: {value.shape}")
    value[:, :, :] = patching_neuron_activation[:, :, :]
    return value

def mlp_new_neuron_hook(
    value: Float[th.Tensor, "batch pos d_mlp"],
    hook: HookPoint
) -> Float[th.Tensor, "batch pos d_mlp"]:
    print(f"Shape of the value tensor: {value.shape}")
    value[:, :, neurons[0]] = original_neuron_activation[:, :, neurons[0]]
    value[:, :, neurons[1]] = original_neuron_activation[:, :, neurons[1]]
    return value

new_logits = model.run_with_hooks(tokens, fwd_hooks=[("blocks.0.mlp.hook_post", mlp_ablation_hook), 
                                                     (f"blocks.{layer}.mlp.hook_post", mlp_new_neuron_hook)])

Shape of the value tensor: torch.Size([1, 14, 2048])
Shape of the value tensor: torch.Size([1, 14, 2048])


In [46]:
original_neuron_activation

tensor([[[-0.0388, -0.1204, -0.0145,  ..., -0.0833,  0.0398, -0.1080],
         [-0.0575, -0.1016, -0.1475,  ..., -0.1400, -0.1365,  0.3035],
         [-0.1224, -0.1228, -0.1077,  ..., -0.1422, -0.1699, -0.1699],
         ...,
         [-0.1521, -0.1035, -0.1437,  ..., -0.1474, -0.1442, -0.1580],
         [-0.1348, -0.1215, -0.1167,  ..., -0.1240, -0.1696,  0.1118],
         [-0.1469, -0.1357, -0.0492,  ..., -0.1335, -0.0162, -0.1569]]],
       device='cuda:0')

In [64]:
# Find the largest difference for each token
diff = new_logits- original_logits
diff = diff.abs()
diff_val, diff_ind = diff.max(dim=-1)
out = model.to_str_tokens(diff_ind)
for o, v in zip(out, diff_val[0]):
    print(repr(o), v.item())
print("======================\n")
val,ind = original_logits.max(dim=-1)
out = model.to_str_tokens(ind)
for o, v in zip(out, val[0]):
    print(repr(o), v.item())

'<|endoftext|>' 0.0
'5' 10.500371932983398
'nd' 10.349782943725586
')].' 13.748017311096191
'ти' 14.573488235473633
'yrs' 14.681735038757324
'idemargin' 14.050003051757812
'ificance' 18.274391174316406
']).' 18.546470642089844
'))$.' 16.930084228515625
')\\].' 15.339709281921387
' 12' 11.824020385742188
'tml' 13.262931823730469
')\\].' 17.10406494140625

'\n' 11.203080177307129
':' 12.916642189025879
'\n' 17.881086349487305
'\n' 16.53376579284668
' is' 15.666515350341797
' the' 16.361787796020508
' difference' 14.399608612060547
' of' 16.922134399414062
' the' 15.69821548461914
'?' 19.01654052734375
'\n' 19.311105728149414
'\n' 16.671977996826172
' is' 15.80677604675293
'\n' 16.598018646240234


In [66]:
val,ind = new_logits.max(dim=-1)
out = model.to_str_tokens(ind)
for o, v in zip(out, val[0]):
    print(repr(o), v.item())

'\n' 11.203080177307129
'.' 13.492734909057617
' 3' 14.281828880310059
' 4' 17.700979232788086
' 5' 17.17980194091797
' 3' 17.17455291748047
' 6' 17.38115882873535
' 8' 19.010574340820312
' 8' 19.93307876586914
' 9' 19.177921295166016
' 7' 17.89773178100586
' 11' 18.138347625732422
' 12' 17.840625762939453
' 12' 17.692947387695312


In [65]:
for o in out:
    print(o)
for v in val:
    print(v)



:




 is
 the
 difference
 of
 the
?




 is


tensor([11.2031, 12.9166, 17.8811, 16.5338, 15.6665, 16.3618, 14.3996, 16.9221,
        15.6982, 19.0165, 19.3111, 16.6720, 15.8068, 16.5980], device='cuda:2',
       grad_fn=<UnbindBackward0>)


In [123]:
from circuitsvis.tokens import colored_tokens
colored_tokens(["hey"], [1.3])